# AlexNet Visualization

现在我们使用一个已经训练完毕的AlexNetModel来实现CNN可视化,我们已经知道CNN的反卷积可视化步骤如下:

CONV==>RELU==>POOL==>UNPOOL==>RELU==>DECONV.

### 1. Load Weights and Bias

首先我们需要提取ALexNet训练好的```weights```和```bais```并用来完成Forward,用以实现反卷积可视化步骤中的:

CONV==>RELU==>POOL==>

```python
    def LoadWandb(self):
        """
        Reloaded weights and bias in save model_dir.
        """
        new_saver = tf.train.import_meta_graph(self.model_dir + '.meta')
        new_saver.restore(self.sess, self.model_dir)
        graph = tf.get_default_graph()
        self.W1 = graph.get_tensor_by_name('W%d:0' % (1))
        self.b1 = graph.get_tensor_by_name('b%d:0' % (1))
        self.W2 = graph.get_tensor_by_name('W%d:0' % (2))
        self.b2 = graph.get_tensor_by_name('b%d:0' % (2))
        self.W3 = graph.get_tensor_by_name('W%d:0' % (3))
        self.b3 = graph.get_tensor_by_name('b%d:0' % (3))
        self.W4 = graph.get_tensor_by_name('W%d:0' % (4))
        self.b4 = graph.get_tensor_by_name('b%d:0' % (4))
        self.W5 = graph.get_tensor_by_name('W%d:0' % (5))
        self.b5 = graph.get_tensor_by_name('b%d:0' % (5))```

```LoadWandb```函数使用给予的model目录来打开一个保存好的网络,并提取```weights```,```bias```,代码很简单就不多阐述了,不明白的朋友回到前面的AlexNet章节查看如何使用Tensorflow保存模型.

现在我们已经拥有了训练好的```weights```,```bias```接着就是构造Forwar,需要注意的是构造Foward不能单纯的直接流动至CONV5,因为我们不仅仅只是查看CONV5的features map.

### 2. Forward

```python
    def forward(self, data, layers='layer1'):
        """
        forward propagation.

        :param data (tensor): training data set,shape [batch_size,227,227,3].
        :param layers (string): control forward step.

        """
        C1 = self._Conv(data, self.W1, self.b1, [1, 4, 4, 1], "VALID", 'Conv_1')
        R1 = tf.nn.relu(C1)
        pooled_1, max_index_1 = self._Pool(R1, [1, 3, 3, 1], [1, 2, 2, 1], 'VALID', 'Pool_1')
        if layers == 'layer1':
            return pooled_1, max_index_1
        C2 = self._Conv(pooled_1, self.W2, self.b2, [1, 1, 1, 1], 'SAME', 'Conv_2')
        R2 = tf.nn.relu(C2)
        pooled_2, max_index_2 = self._Pool(R2, [1, 3, 3, 1], [1, 2, 2, 1], 'VALID', 'Pool_2')
        if layers == 'layer2':
            return pooled_2, max_index_2, max_index_1
        C3 = self._Conv(pooled_2, self.W3, self.b3, [1, 1, 1, 1], 'SAME', 'Conv_3')
        R3 = tf.nn.relu(C3)
        if layers == 'layer3':
            return R3, max_index_2, max_index_1
        C4 = self._Conv(R3, self.W4, self.b4, [1, 1, 1, 1], 'SAME', 'Conv_4')
        R4 = tf.nn.relu(C4)
        if layers == 'layer4':
            return R4, max_index_2, max_index_1
        C5 = self._Conv(R4, self.W5, self.b5, [1, 1, 1, 1], 'SAME', 'Conv_5')
        R5 = tf.nn.relu(C5)
        pooled_5, max_index_5 = self._Pool(R5, [1, 3, 3, 1], [1, 2, 2, 1], 'VALID', 'Pool_5')
        if layers == 'layer5':
            return pooled_5, max_index_5, max_index_2, max_index_1```

比如我们要查看CONV1的features map Visualization,那么我们的过程如下:

```Input==>C1==>R1==>P1==>Unpool1(P1,ind1)==>UnR1==>deCONV1==>Input```.

如果我们要查看CONV2的features map Visualization,那么我们的过程如下:

```Input==>C1==>R1==>P1==>C2==>R2==>P2==>Unpool2(P2,ind2)
==>UnR2==>deCONV2==>Unpool1(deCONV2,ind1)==>UnR1==>DeCONV1==>Input```.

CONV3,CONV4,CONV5也是如此,不过需要注意的是,CONV3和CONV4没有Pool操作,当然也就不需要Unpool.

所以我们的```forward```函数参数```layer```来控制我们需要查看可视化结果的层数,以及返回必要参数.

### 3. DeConv

当Forward设置完毕之后,我们需要建立DeConv,我们知道反卷积网络实际上是一个重复的过程:

比如我们要查看CONV3,那么Forward至CONV3后,进行```DeConv3==>DeConv2==>DeConv1==>OutPut```.

比如我们要查看CONV4,那么Forward至CONV4后,进行```DeConv4==>DeConv3==>DeConv2==>DeConv1==>OutPut```

```python

    def deconv1(self, pooled_1, max_index_1, m):
        C_transpose_1 = self._Deconv_hanlder(pooled_1, max_index_1, [m, 55, 55, 96],
                                             self.W1, [m, 227, 227, 3], [1, 4, 4, 1], "VALID")
        return C_transpose_1

    def deconv2(self, pooled_2, max_index_2, max_index_1, m):
        C_transpose_2 = self._Deconv_hanlder(pooled_2, max_index_2, [m, 27, 27, 256],
                                             self.W2, [m, 27, 27, 96], [1, 1, 1, 1], 'SAME')
        normal = self.deconv1(C_transpose_2, max_index_1, m)
        return normal

    def deconv3(self, R3, max_index_2, max_index_1, m):
        C_transpose_3 = tf.nn.conv2d_transpose(R3, self.W3, [m, 13, 13, 256], [1, 1, 1, 1], "SAME")
        normal = self.deconv2(C_transpose_3, max_index_2, max_index_1, m)
        return normal

    def deconv4(self, R4, max_index_2, max_index_1, m):
        C_transpose_4 = tf.nn.conv2d_transpose(R4, self.W4, [m, 13, 13, 384], [1, 1, 1, 1], "SAME")
        normal = self.deconv3(C_transpose_4, max_index_2, max_index_1, m)
        return normal

    def deconv5(self, pooled_5, max_index_5, max_index_2, max_index_1, m):
        C_transpose_5 = self._Deconv_hanlder(pooled_5, max_index_5, [m, 13, 13, 256],
                                             self.W5, [m, 13, 13, 384], [1, 1, 1, 1], 'SAME')
        normal = self.deconv4(C_transpose_5, max_index_2, max_index_1, m)
        return normal```

所以我们创建以上代码来实现DeConv:

比如需要查看CONV2的时候,那么```deconv2```会先实现自身的deconv,再将结果流入到```deconv1```.

比如需要查看CONV3的时候,那么```deconv3```会先实现自身的deconv,再将结果流入到```deconv2```,而```deconv2```又会计算自身的deconv结果进而流入到```deconv1```.

这样我们就能够实现DeConv了,现在总结一下:

1. Forward 会返回目标layer的features map以及必要参数比如```ind```或者```pooled```.

2. DeConv 接受到Forward的结果进行反卷积可视化,最后都是返回DeConv1的结果.

实现代码如下:

```python
    def _Deconv(self, layer='layer1'):
        """
        deconv...
        :param layer (string): control deConvolution layers.[layer1-layer5]
        :return
            normal (tensor): deconv result. shape [batch_size,227,227,3]
            images (ndarray): original images. shape [batch_size,227,227,3]
        """
        next_element_train = self.load_images()
        images, _ = self.sess.run(next_element_train)

        m = images.shape[0]
        if layer == 'layer1':
            pooled_1, max_index_1 = self.forward(images, layers=layer)
            normal = self.deconv1(pooled_1, max_index_1, m)
            return normal, images

        if layer == 'layer2':
            pooled_2, max_index_2, max_index_1 = self.forward(images, layers=layer)
            normal = self.deconv2(pooled_2, max_index_2, max_index_1, m)
            return normal, images
        if layer == 'layer3':
            R3, max_index_2, max_index_1 = self.forward(images, layer)
            normal = self.deconv3(R3, max_index_2, max_index_1, m)
            return normal, images
        if layer == 'layer4':
            R4, max_index_2, max_index_1 = self.forward(images, layer)
            normal = self.deconv4(R4, max_index_2, max_index_1, m)
            return normal, images

        if layer == 'layer5':
            pooled_5, max_index_5, max_index_2, max_index_1 = self.forward(images, layer)
            normal = self.deconv5(pooled_5, max_index_5, max_index_2, max_index_1, m)
            return normal, images```

最后将```DeConv1```中的结果经过Normal(std,range)处理一下就可以使用matplotlib绘制了.

完整代码查看[AlexNetDeConv](DeConvlution.py)

### 4. Visualization

下面是可视化的结果:

<table border="0">
	<tbody>
    <tr>
			<td  align="center"><h3>Layer 1</h3></td>
			<td > <img src=" visresult/Original_layer1.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer1.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 2</h3></td>
			<td > <img src=" visresult/Original_layer2.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer2.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 3</h3></td>
			<td > <img src=" visresult/Original_layer3.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer3.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 4</h3></td>
			<td > <img src=" visresult/Original_layer4.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer4.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 5</h3></td>
			<td > <img src=" visresult/Original_layer5.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer5.jpg" style="max-width:100%;"></td>
	</tr>
	</tbody>
</table>

(1) 首先对于DeCONV1而言,可视化结果比较清楚,也没有太多的照声,说明模型是收敛的,实际上我们也可以使用DeConv1来观察模型是否可行,对于小样本集(比如100个samples)如果噪声太多,那么模型没有必要在大数据下进行训练,因为百分之90都是在做无用功.

(2) 对于DeCONV2而言,模型貌似学习到动物的轮廓,可以很明显的看出每张图片中动物的轮廓比较清晰.
 
(3) 对于DeCONV3而言,模型貌似学习到动物的眼睛,比如第二行第三列,可以清楚的看到猫的眼睛像素值较高.

(4) 对于DeCONV4而言,模型貌似学习到动物的位置,或者是背景,或者是图片中最高亮的部分,不是能很清楚地观察出来.

(5) 对于DeCONV4而言,模型貌似学习到动物的面容,比较清楚的是最后一行的最后一列和第一行的第一列.

接下去我们来看看Mask.

### 5. Mask

我们知道ZFNet中对于mask也有阐述,通过掩盖图片不同的位置,模型预测结果会受到不同程度的影响,也就是说**整个模型是否知道目标在图像中的具体位置**.

需要注意的是,我们在进行mask的时候,掩盖的地方不能完全和paper一样,因为我们网络各层学到的东西不一定和paper一样,所以我们是需要掩盖我们模型各层学到的部分,比如眼睛(DeCONV3),比如面部(DeConv5).

另外需要注意的一点是,在选择掩模图片时,我们不能选择噪声太小的图片(也就是说不能整张图片全是猫或者狗),因为如果选择噪声太小的图片,即使进行掩模后,输入的图片($224,224$)特征也几乎是完整的,没有达到破坏原图特征的结果.举一个简单的例子:

![](MASK/cat.jpg)

可以看到,红色是模型输入的$(224,224)$,那么特征依然较为完全,我们CONV2的轮廓,CONV5的面部的特征都还是比较完整的,这样就可能达不到我们的目的.

#### 5.1 past gray scale

现在我们即可创建一个mask的函数,使用的是```PIL```库,如果有朋友看不懂代码的,那么直接使用PS处理需要掩盖的部分也是一样的.

```python
def mask(file_path, SizeBoxsName, Savedir='MASK'):
    """
    mask image.
    :param file_path (string): picture's path.
    :param SizeBoxsName (tuple):Include 3 params:
                                 1.Size (tuple): pasted image size (h,w,c). channels must be equal original image.
                                 2.Box (tuple): pasted position.accept 4-D(left,top,right,bottom) or 2-D(let,top).
                                 3.Name (string): mask position name also it's a save file name.like "dog_eyes".
    :param Savedir (string): save dir,if not given,default save in current path "MASK" dir.
    """

    for size, box, name in SizeBoxsName:
        img = Image.open(file_path)
        sub_img = Image.fromarray(np.uint8(100 * np.ones(size)))
        img.paste(sub_img, box)

        if not os.path.exists(Savedir):
            os.mkdir(Savedir)
        path = os.path.join(Savedir, name + '.jpg')
        img.save(path)```

[img.paste](https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.paste)提供掩盖功能,另外我们使用的是灰色掩盖(像素值在100-130左右),所以我们乘上100.请注意,我们的掩盖操作实际上是不希望模型**"看到"某些部分**,所以我们的像素值不能选择的太大,因为模型的运算实际上还是**运算像素值**,如果太大,那么该像素特征就会被纳入到最后的输出从而也就变成了模型**能够"看到"某些部分**.

下面我们使用```mask```函数并且使用AlexNet章节中的```Predict```函数得到以下结果:

<table border="0">
	<tbody>
    <tr>
			<td style="text-align: center"><strong>Original</strong></td>
			<td style="text-align: center"><strong>Eyes</strong></td>
            <td style="text-align: center"><strong>Face</strong></td>
            <td style="text-align: center"><strong>Other</strong></td>
	</tr>
    <tr>
			<td > <img src="MASK/dog.jpg" style="max-width:100%;"></td>
			<td > <img src="MASK/dog_eyes.jpg" style="max-width:100%;"></td>
            <td > <img src="MASK/dog_face.jpg" style="max-width:100%;"></td>
            <td > <img src="MASK/dog_other.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td style="text-align: center"><strong>The picture is cat rate: 0.0000 % <br> The picture is dog rate: 100.0000 %</strong></td>
			<td style="text-align: center"><strong>The picture is cat rate: 10.0000 % <br>
    The picture is dog rate: 90.0000 % </strong></td>
            <td style="text-align: center"><strong>The picture is cat rate: 70.0000 % <br>
    The picture is dog rate: 30.0000 %</strong></td>
            <td style="text-align: center"><strong>The picture is cat rate: 0.0000 % <br>
    The picture is dog rate: 100.0000 % </strong></td>
	</tr>
	</tbody>
</table>

可以看出原图的预测是正确的.

当我们掩盖了眼睛(DeConv2),输出的正确率发生了变化,但是分类结果依然是正确的,说明掩盖眼睛确实对模型是有影响的.

当我们掩盖了面部(DeConv5),输出的结果也发生了变化,并且分类错误,说明脸部是CONV层中最重要的部分.

当我们掩盖了其他部分(也就是无关紧要的地方),分类结果不会发生任何改变.

所以模型确实知道物体的具体位置,以及知道哪些位置是比较重要的.